<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=28cd52061a73e5d911f63aab3835dd38fc1d9ead9b0dcef01a90e755399a3b4d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-02 15:42:39
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 84.05 K (0.59%)
Current PnL: -24.72 L (-15.76%)
CY Booked + Current PnL: -10.43 L (-6.65%)
-------------------
Total profit:  1.89 L
Total loss:  -26.61 L
-------------------
Total Booked + Current PnL: 16.55 L (12.75%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.9%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.69 L (62.13%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,3.32,11.88,6.98,19.69,14974.0,22788.0,214530.0,0.99,63.0,X-MC,2.46,78.0,1.52,1.49,19.49,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.15,-20.16,25.37,0.09,20413.0,-20317.0,80460.0,86.77,37.0,M-SC,3.49,253.0,-1.00,0.56,5.71,OX40N,NTT,DURABLES
17,COALINDIA,484.83,6.85,5.23,13.30,19.23,21057.0,7862.0,158323.0,22.81,83.0,L-LC,4.63,179.0,0.37,1.10,27.61,XY25,ATH,MINING
49,MASFIN,398.61,1.78,-0.40,22.54,22.05,21997.0,-390.0,97590.0,-14.18,57.0,H-SC,6.52,164.0,-0.02,0.68,41.85,XR,ATH,FINANCE
51,NESTLEIND,1377.00,-1.18,12.82,7.60,21.40,22758.0,34024.0,299450.0,4.67,59.0,X-LC,5.06,12.0,1.50,2.07,19.96,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,6.85,5.23,13.30,19.23,21057.0,7862.0,158323.0,22.81,83.0,L-LC,4.63,179.0,0.37,1.10,27.61,XY25,ATH,MINING
20,DABUR,735.00,4.53,2.17,40.64,43.70,103007.0,5393.0,253461.0,-6.31,68.0,X-MC,1.91,72.0,0.05,1.76,17.55,XY24,BTT,FMCG
57,RECLTD,446.00,3.52,3.05,17.17,20.74,35947.0,6188.0,209358.0,35.83,73.0,M-MC,7.89,189.0,0.17,1.45,13.98,XY25,NTT,FINANCE
82,VOLTAS,1530.00,3.32,11.88,6.98,19.69,14974.0,22788.0,214530.0,0.99,63.0,X-MC,2.46,78.0,1.52,1.49,19.49,XY25,NTT,AC
54,QUESS,424.00,3.22,-27.55,97.17,42.84,45761.0,-17912.0,47094.0,-52.62,53.0,X-SC,30.45,83.0,-0.39,0.33,6.55,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-3.79,-14.21,29.12,10.78,60039.0,-34139.0,206179.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
0,5PAISA,593.0,-2.13,-29.97,61.23,12.90,93685.0,-65491.0,153005.0,125.96,67.0,H-SC,11.47,161.0,-0.70,1.06,27.22,OX40N,NTT,FINANCE
6,ATULAUTO,844.0,-2.05,-19.96,81.86,45.57,129170.0,-39338.0,157794.0,3413.51,59.0,M-SC,6.29,244.0,-0.30,1.09,11.87,XY24,NTT,AUTO
12,BERGEPAINT,680.0,-1.79,-6.63,29.75,21.14,63147.0,-15079.0,212260.0,-13.17,38.0,X-MC,2.04,74.0,-0.24,1.47,17.88,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,-1.61,1.49,18.99,20.77,45127.0,3491.0,237635.0,-7.31,40.0,X-MC,1.28,62.0,0.08,1.65,8.51,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.24,-0.64,109.26,107.91,160276.0,-952.0,146692.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.38,OX40N,ATH,CABLES
49,MASFIN,398.61,1.78,-0.40,22.54,22.05,21997.0,-390.0,97590.0,-14.18,57.0,H-SC,6.52,164.0,-0.02,0.68,41.85,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.24,-0.64,109.26,107.91,160276.0,-952.0,146692.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.38,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.33,-16.80,20.22,0.02,29365.0,-29330.0,145229.0,121.54,47.0,M-SC,6.78,234.0,-1.00,1.01,26.32,OX40N,NTT,PAINTS
47,KANSAINER,340.00,0.57,-19.54,41.03,13.47,89024.0,-52695.0,216972.0,-66.95,61.0,H-SC,6.03,158.0,-0.59,1.50,10.32,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.15,-20.16,25.37,0.09,20413.0,-20317.0,80460.0,86.77,37.0,M-SC,3.49,253.0,-1.00,0.56,5.71,OX40N,NTT,DURABLES
66,SIS,528.00,2.37,-22.07,55.61,21.26,48115.0,-24510.0,86522.0,2044.04,59.0,H-SC,4.59,166.0,-0.51,0.60,16.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,3.52,3.05,17.17,20.74,35947.0,6188.0,209358.0,35.83,73.0,M-MC,7.89,189.0,0.17,1.45,13.98,XY25,NTT,FINANCE
17,COALINDIA,484.83,6.85,5.23,13.30,19.23,21057.0,7862.0,158323.0,22.81,83.0,L-LC,4.63,179.0,0.37,1.10,27.61,XY25,ATH,MINING
84,WIPRO,420.00,0.62,10.33,56.13,72.27,104938.0,17507.0,186955.0,-5.17,65.0,M-LC,5.62,99.0,0.17,1.30,17.14,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.62,10.33,56.13,72.27,104938.0,17507.0,186955.0,-5.17,65.0,M-LC,5.62,99.0,0.17,1.30,17.14,XR,NTT,IT
25,FINCABLES,1641.55,-0.24,-0.64,109.26,107.91,160276.0,-952.0,146692.0,-19.34,57.0,M-SC,11.04,216.0,-0.01,1.02,9.38,OX40N,ATH,CABLES
49,MASFIN,398.61,1.78,-0.40,22.54,22.05,21997.0,-390.0,97590.0,-14.18,57.0,H-SC,6.52,164.0,-0.02,0.68,41.85,XR,ATH,FINANCE
38,INDIAMART,4810.62,-0.74,-6.42,120.89,106.72,139535.0,-7913.0,115423.0,-53.68,37.0,H-SC,6.46,139.0,-0.06,0.80,15.46,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.20,-3.31,40.27,35.63,81601.0,-6939.0,202635.0,-16.74,46.0,H-MC,2.89,119.0,-0.09,1.40,12.81,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-3.79,-14.21,29.12,10.78,60039.0,-34139.0,206179.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
23,DMART,5391.45,0.10,-10.58,44.94,29.60,100301.0,-26412.0,223188.0,-23.73,32.0,X-LC,9.15,19.0,-0.26,1.55,9.28,X40N,ATH,FMCG
22,DIXON,18931.72,0.61,-15.66,55.62,31.26,108259.0,-36128.0,194640.0,-60.59,33.0,X-MC,17.06,56.0,-0.33,1.35,3.38,X40N,ATH,IT
7,AWL,485.00,-1.33,-23.42,105.27,57.19,262898.0,-76390.0,249737.0,-63.76,35.0,X-MC,7.37,58.0,-0.29,1.73,0.22,XY24,NTT,FMCG
52,PAGEIND,51605.07,0.36,-9.45,44.25,30.62,94983.0,-22404.0,214650.0,-35.61,36.0,X-MC,15.49,55.0,-0.24,1.49,0.59,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.21,-16.77,50.81,25.51,78697.0,-31210.0,154885.0,-4.98,45.0,H-LC,1.17,49.0,-0.40,1.07,14.14,AR,ATH,ELECTRICAL
78,UNITDSPR,1644.0,-1.61,1.49,18.99,20.77,45127.0,3491.0,237635.0,-7.31,40.0,X-MC,1.28,62.0,0.08,1.65,8.51,X40N,NTT,BREWERIES
20,DABUR,735.0,4.53,2.17,40.64,43.70,103007.0,5393.0,253461.0,-6.31,68.0,X-MC,1.91,72.0,0.05,1.76,17.55,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-1.79,-6.63,29.75,21.14,63147.0,-15079.0,212260.0,-13.17,38.0,X-MC,2.04,74.0,-0.24,1.47,17.88,XY24,NTT,PAINTS
82,VOLTAS,1530.0,3.32,11.88,6.98,19.69,14974.0,22788.0,214530.0,0.99,63.0,X-MC,2.46,78.0,1.52,1.49,19.49,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-3.79,-14.21,29.12,10.78,60039.0,-34139.0,206179.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.33,-23.42,105.27,57.19,262898.0,-76390.0,249737.0,-63.76,35.0,X-MC,7.37,58.0,-0.29,1.73,0.22,XY24,NTT,FMCG
52,PAGEIND,51605.07,0.36,-9.45,44.25,30.62,94983.0,-22404.0,214650.0,-35.61,36.0,X-MC,15.49,55.0,-0.24,1.49,0.59,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.36,-38.71,120.60,35.20,95108.0,-49808.0,78862.0,3.39,40.0,X-SC,17.33,92.0,-0.52,0.55,1.29,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.48,-26.51,123.35,64.15,215714.0,-63071.0,174880.0,-57.19,41.0,X-SC,5.88,82.0,-0.29,1.21,1.37,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,3.22,-27.55,97.17,42.84,45761.0,-17912.0,47094.0,-52.62,53.0,X-SC,30.45,83.0,-0.39,0.33,6.55,XY24,NTT,MISC
58,RELAXO,1176.00,0.16,-47.77,191.59,52.31,145264.0,-69340.0,75820.0,-44.34,45.0,X-SC,7.48,91.0,-0.48,0.53,1.49,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.36,-38.71,120.60,35.20,95108.0,-49808.0,78862.0,3.39,40.0,X-SC,17.33,92.0,-0.52,0.55,1.29,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.11,1.63,22.09,24.08,28499.0,2063.0,129013.0,-6.61,58.0,X-SC,7.42,89.0,0.07,0.89,21.56,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.74,-17.42,73.55,43.32,98925.0,-28372.0,134500.0,-29.05,45.0,X-SC,7.66,90.0,-0.29,0.93,3.27,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.72,-10.62,35.05,20.71,118495.0,-40164.0,338073.0,-22.98,55.0,X-LC,2.82,1.0,-0.34,2.34,12.96,X40,ATH,IT
41,INFY,1972.00,0.65,11.67,20.21,34.24,70615.0,36517.0,349405.0,-13.14,58.0,X-LC,7.43,2.0,0.52,2.42,20.92,X40,NTT,IT
75,TMPV,600.00,0.76,-24.09,62.01,22.98,106330.0,-54426.0,171472.0,-21.86,62.0,X-LC,5.50,3.0,-0.51,1.19,7.85,XY24,NTT,AUTO
81,VBL,671.64,0.42,-0.25,36.01,35.67,113447.0,-798.0,315044.0,-12.17,65.0,X-LC,2.67,4.0,-0.01,2.18,13.57,X40N,ATH,FMCG
43,ITC,452.00,-3.79,-14.21,29.12,10.78,60039.0,-34139.0,206179.0,-48.92,15.0,X-LC,3.14,5.0,-0.57,1.43,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.38,-33.37,99.46,32.89,51162.0,-25766.0,51440.0,-735.55,71.0,L-MC,6.66,259.0,-0.50,0.36,41.66,XR,NTT,BANKS
49,MASFIN,398.61,1.78,-0.40,22.54,22.05,21997.0,-390.0,97590.0,-14.18,57.0,H-SC,6.52,164.0,-0.02,0.68,41.85,XR,ATH,FINANCE
13,BSOFT,831.70,0.01,-21.67,92.50,50.79,101910.0,-30480.0,110173.0,0.50,54.0,H-SC,10.81,151.0,-0.30,0.76,28.28,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.33,-16.80,20.22,0.02,29365.0,-29330.0,145229.0,121.54,47.0,M-SC,6.78,234.0,-1.00,1.01,26.32,OX40N,NTT,PAINTS
69,SURYODAY,216.00,1.62,-18.00,50.04,23.03,73478.0,-32232.0,146839.0,60.09,58.0,H-SC,5.70,167.0,-0.44,1.02,45.69,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,6.85,5.23,13.30,19.23,21057.0,7862.0,158323.0,22.81,83.0,L-LC,4.63,179.0,0.37,1.10,27.61,XY25,ATH,MINING
74,TITAGARH,1548.00,0.98,-16.18,73.86,45.74,149936.0,-39177.0,203000.0,11.66,63.0,H-SC,3.95,174.0,-0.26,1.41,32.03,XY24,NTT,ENGINEERING
36,ICICIPRULI,790.00,0.59,12.24,16.47,30.73,36419.0,24121.0,221126.0,-14.04,74.0,X-MC,5.04,75.0,0.66,1.53,26.57,X40,ATH,INSURANCE
0,5PAISA,593.00,-2.13,-29.97,61.23,12.90,93685.0,-65491.0,153005.0,125.96,67.0,H-SC,11.47,161.0,-0.70,1.06,27.22,OX40N,NTT,FINANCE
84,WIPRO,420.00,0.62,10.33,56.13,72.27,104938.0,17507.0,186955.0,-5.17,65.0,M-LC,5.62,99.0,0.17,1.30,17.14,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.03
1,25,44.05
2,50,76.37


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.90
MC    30.83
LC    26.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.75
X40      23.77
X40N     12.81
XY25     10.61
XR        9.01
AR        8.59
OX40N     7.50
SR        0.98
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.32
H-SC    22.83
X-LC    22.82
M-SC    11.38
X-SC     7.88
H-MC     4.70
M-MC     1.45
M-LC     1.30
L-LC     1.10
H-LC     1.07
L-SC     0.81
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.93,-7.80,43.16
IT,12.79,-15.75,74.14
FINANCE,11.10,-11.44,58.21
MISC,7.01,-28.46,81.01
ELECTRICAL,5.82,-11.52,52.37
PAINTS,5.67,-12.81,29.57
INSURANCE,4.71,0.33,33.78
PHARMA,3.95,-3.09,35.54
AUTO,2.87,-26.50,70.36


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3108328.0,21
AR,1332092.0,10
XR,1191927.0,12
X40,1089581.0,15
X40N,894817.0,9
OX40N,710067.0,10
XY25,362222.0,7
SR,279590.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3499865.0,24
X-MC,1475750.0,16
M-SC,1415790.0,15
X-LC,929492.0,13
X-SC,783080.0,8
H-MC,399754.0,3
L-SC,173092.0,2
M-LC,104938.0,1
H-LC,78697.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219245.0      6
           AR         914975.0      5
M-SC       XY24       803203.0      6
H-SC       XR         774625.0      7
X-MC       X40        495006.0      7
           XY24       429052.0      3
X-LC       X40        409496.0      6
X-MC       X40N       373042.0      4
H-SC       OX40N      311430.0      4
M-SC       OX40N      309332.0      5
X-SC       X40N       308027.0      3
           XY24       289974.0      3
H-SC       SR         279590.0      2
X-LC       X40N       213748.0      2
H-MC       AR         212580.0      2
           XY24       187174.0      1
X-SC       X40        185079.0      2
X-LC       XY24       179680.0      2
X-MC       XY25       178650.0      2
M-SC       XR         177415.0      2
X-LC       XY25       126568.0      3
M-SC       AR         125840.0      2
M-LC       XR         104938.0      1
L-SC       OX40N       89305.0      1
           XR          83787.0      1
H-LC       AR          78697.0      1
L-MC       XR          51162.0      1
M-MC       XY25        35947.0      1
L-LC       XY25        21057.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
